In [12]:
import pandas as pd
import re
import plotly.graph_objects as go
import datetime
import csv
import numpy as np
from datetime import datetime
import plotly.express as px

In [13]:
# Funkcja do usunięcia dwukropka na końcu wartości
def remove_last_character(value):
    pattern = r':$'  # Wzorzec do dopasowania ostatniego dwukropka
    return re.sub(pattern, '', value)

# Funkcja do konwersji liczby sekund na format hh:mm:ss
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f'{int(hours)}:{int(minutes):02d}:{int(seconds):02d}'

# Wczytanie danych z pliku CSV
data = pd.read_csv('women_marathon.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie dwukropka na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na timedelta
data['Time'] = pd.to_timedelta(data['Time'])

# Dodanie nowej kolumny 'Year' na podstawie kolumny 'Event Date'
data['Year'] = data['Event Date'].dt.year

# Filtrowanie danych od roku 1975
data = data[data['Year'] >= 1975]

In [14]:
data_help = data.copy()
best_results_names = data_help.groupby('Year').apply(lambda x: x.loc[x['Time'].idxmin(), ['Time', 'Athlete']])
print(best_results_names)

                  Time                Athlete
Year                                         
1979.0 0 days 02:27:33            Grete Waitz
1980.0 0 days 02:25:42            Grete Waitz
1981.0 0 days 02:26:46            Allison Roe
1982.0 0 days 02:26:11            Joan Benoit
1983.0 0 days 02:22:43            Joan Benoit
1984.0 0 days 02:24:26     Ingrid Kristiansen
1985.0 0 days 02:21:06     Ingrid Kristiansen
1986.0 0 days 02:24:54            Grete Waitz
1987.0 0 days 02:22:48     Ingrid Kristiansen
1988.0 0 days 02:23:51           Lisa Ondieki
1989.0 0 days 02:24:33     Ingrid Kristiansen
1990.0 0 days 02:25:24              Rosa Mota
1991.0 0 days 02:24:18           Wanda Panfil
1992.0 0 days 02:23:43           Olga Markova
1993.0 0 days 02:24:07            Wang Junxia
1994.0 0 days 02:21:45             Uta Pippig
1995.0 0 days 02:25:11             Uta Pippig
1996.0 0 days 02:26:04           Katrin Dörre
1997.0 0 days 02:22:07          Tegla Loroupe
1998.0 0 days 02:20:47          Te

In [15]:
top_runner_names_each_year = best_results_names['Athlete'].unique().tolist()
top_runner_names_each_year

['Grete Waitz',
 'Allison Roe',
 'Joan Benoit',
 'Ingrid Kristiansen',
 'Lisa Ondieki',
 'Rosa Mota',
 'Wanda Panfil',
 'Olga Markova',
 'Wang Junxia',
 'Uta Pippig',
 'Katrin Dörre',
 'Tegla Loroupe',
 "Catherine N'dereba",
 'Paula Radcliffe',
 'Yoko Shibui',
 'Deena Kastor',
 'Zhou Chunxiu',
 'Irina Mikitenko',
 'Atsede Baysa',
 'Mary Keitany',
 'Rita Sitienei Jeptoo',
 'Gladys Cherono',
 'Beyenne Tsegaye Tirfi',
 'Brigid Kosgei',
 'Peres Jepchirchir',
 'Joyciline Jepkosgei',
 'Ruth Chepngetich',
 'Rosemary Wanjiru']

In [16]:
color_names = [
    "AliceBlue", "AntiqueWhite", "Aqua", "Aquamarine", "Azure", "Beige",
    "Bisque", "Black", "BlanchedAlmond", "Blue", "BlueViolet", "Brown",
    "BurlyWood", "CadetBlue", "Chartreuse", "Chocolate", "Coral", "CornflowerBlue",
    "Cornsilk", "Crimson", "Cyan", "DarkBlue", "DarkCyan", "DarkGoldenRod",
    "DarkGray", "DarkGreen", "DarkKhaki", "DarkMagenta", "DarkOliveGreen",
    "DarkOrange", "DarkOrchid", "DarkRed", "DarkSalmon", "DarkSeaGreen",
    "DarkSlateBlue", "DarkSlateGray", "DarkTurquoise", "DarkViolet", "DeepPink",
    "DeepSkyBlue", "DimGray", "DodgerBlue", "FireBrick", "FloralWhite",
    "ForestGreen", "Fuchsia", "Gainsboro", "GhostWhite", "Gold", "GoldenRod",
    "Gray", "Green", "GreenYellow", "HoneyDew", "HotPink", "IndianRed", "Indigo",
    "Ivory", "Khaki", "Lavender", "LavenderBlush", "LawnGreen", "LemonChiffon",
    "LightBlue", "LightCoral", "LightCyan", "LightGoldenRodYellow", "LightGray",
    "LightGreen", "LightPink", "LightSalmon", "LightSeaGreen", "LightSkyBlue",
    "LightSlateGray", "LightSteelBlue", "LightYellow", "Lime", "LimeGreen",
    "Linen", "Magenta", "Maroon", "MediumAquaMarine", "MediumBlue", "MediumOrchid",
    "MediumPurple", "MediumSeaGreen", "MediumSlateBlue", "MediumSpringGreen",
    "MediumTurquoise", "MediumVioletRed", "MidnightBlue", "MintCream", "MistyRose",
    "Moccasin", "NavajoWhite", "Navy", "OldLace", "Olive", "OliveDrab",
    "Orange", "OrangeRed", "Orchid", "PaleGoldenRod", "PaleGreen", "PaleTurquoise",
    "PaleVioletRed", "PapayaWhip", "PeachPuff", "Peru", "Pink", "Plum",
    "PowderBlue", "Purple", "Red", "RosyBrown", "RoyalBlue", "SaddleBrown",
    "Salmon", "SandyBrown", "SeaGreen", "SeaShell", "Sienna", "Silver",
    "SkyBlue", "SlateBlue", "SlateGray", "Snow", "SpringGreen", "SteelBlue",
    "Tan", "Teal", "Thistle", "Tomato", "Turquoise", "Violet", "Wheat", "White",
    "WhiteSmoke", "Yellow", "YellowGreen"
]


In [17]:
athlete_colors = dict(zip(top_runner_names_each_year, color_names[:len(top_runner_names_each_year)]))
athlete_colors

{'Grete Waitz': 'AliceBlue',
 'Allison Roe': 'AntiqueWhite',
 'Joan Benoit': 'Aqua',
 'Ingrid Kristiansen': 'Aquamarine',
 'Lisa Ondieki': 'Azure',
 'Rosa Mota': 'Beige',
 'Wanda Panfil': 'Bisque',
 'Olga Markova': 'Black',
 'Wang Junxia': 'BlanchedAlmond',
 'Uta Pippig': 'Blue',
 'Katrin Dörre': 'BlueViolet',
 'Tegla Loroupe': 'Brown',
 "Catherine N'dereba": 'BurlyWood',
 'Paula Radcliffe': 'CadetBlue',
 'Yoko Shibui': 'Chartreuse',
 'Deena Kastor': 'Chocolate',
 'Zhou Chunxiu': 'Coral',
 'Irina Mikitenko': 'CornflowerBlue',
 'Atsede Baysa': 'Cornsilk',
 'Mary Keitany': 'Crimson',
 'Rita Sitienei Jeptoo': 'Cyan',
 'Gladys Cherono': 'DarkBlue',
 'Beyenne Tsegaye Tirfi': 'DarkCyan',
 'Brigid Kosgei': 'DarkGoldenRod',
 'Peres Jepchirchir': 'DarkGray',
 'Joyciline Jepkosgei': 'DarkGreen',
 'Ruth Chepngetich': 'DarkKhaki',
 'Rosemary Wanjiru': 'DarkMagenta'}

In [18]:
# Grupowanie danych według roku i wybór najlepszego wyniku dla każdego roku
best_results = data.groupby('Year')['Time'].min().reset_index()

# Wyszukanie najlepszych zawodników dla każdego roku
best_athletes = data.merge(best_results, on=['Year', 'Time'])

In [19]:
# Tworzenie wykresu za pomocą Plotly
fig = go.Figure()

# Dodawanie punktów na wykresie dla każdego zawodnika
for index, row in best_athletes.iterrows():
    athlete = row['Athlete']
    year = row['Year']
    time = row['Time']
    country = row['Country']
    time_str = format_time(time.total_seconds())
    label = f"Zawodniczka: {athlete}<br>Rok: {int(year)}<br>Czas: {time_str}<br>Kraj: {country}"
    color = athlete_colors.get(athlete, 'blue')  # Pobieranie koloru dla zawodnika, domyślnie 'blue'
    fig.add_trace(go.Scatter(
        x=[year],
        y=[time.total_seconds()],
        mode='markers',
        name=athlete,
        text=[label],
        hovertemplate='%{text}<extra></extra>',
        marker=dict(size=15, color=color),
    ))

# Konfiguracja układu wykresu
fig.update_layout(
    xaxis=dict(title='Rok'),
    yaxis=dict(
        title='Najlepszy czas',
        tickvals=[8900, 8750, 8550, 8350, 8150, 8000],  # Wartości osi Y
        ticktext=['2:27:00', '2:24:00', '2:22:00', '2:20:00', '2:18:00', '2:16:00'],  # Etykiety osi Y
    ),
    title='Najlepszy wynik w maratonie (HH:MM:SS)',
    showlegend=True,
    height= 600,
)

# Zapisywanie wykresu do pliku HTML
fig.write_html('best_of_every_season.html')
fig.show()



In [20]:
def calculate_age(birth_date, event_date):
    birth_date = datetime.strptime(birth_date, "%d.%m.%y")
    event_date = datetime.strptime(event_date, "%d.%m.%Y")
    
    birth_year = birth_date.year
    event_year = event_date.year
    
    if birth_year > 2010:
        birth_year = birth_year - 100
    
    age = event_year - birth_year
    if event_date.month < birth_date.month or (event_date.month == birth_date.month and event_date.day < birth_date.day):
        age -= 1
    return age


def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)


# Wczytanie danych z pliku CSV
data = pd.read_csv('women_marathon.csv')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.split(':').apply(lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))

# Znalezienie najlepszego czasu dla każdego zawodnika
best_times = data.groupby('Athlete')['Time'].min()

all_ages = []
# Iteracja po zawodnikach i ich najlepszych czasach
for athlete, best_time in best_times.items():
    athlete_data = data[(data['Athlete'] == athlete) & (data['Time'] == best_time)]
    
    birth_date = athlete_data['Date of Birth'].values[0]
    event_date = athlete_data['Event Date'].values[0]

    # Sprawdzenie, czy birth_date lub event_date jest puste lub nie jest prawidłową datą
    if pd.isnull(birth_date) or pd.isnull(event_date) or not isinstance(birth_date, str) or not isinstance(event_date, str):
        continue
    
    try:
        age = calculate_age(birth_date, event_date)
        all_ages.append(age)
    except ValueError:
        continue

unique_values, frequencies = np.unique(all_ages, return_counts=True)
value_to_delete = 83

# Znalezienie indeksu wartości do usunięcia
index_to_delete = np.where(unique_values == value_to_delete)[0]
unique_values = np.delete(unique_values, index_to_delete)
frequencies = np.delete(frequencies, index_to_delete)

# Tworzenie wykresu w Plotly
fig = go.Figure()

# Dodawanie słupków na wykres
fig.add_trace(go.Bar(
    x=unique_values,
    y=frequencies,
    marker_color='#4287f5',  # Kolor słupków
    text=frequencies,
    textposition='outside',  # Wyświetlanie tekstu nad słupkiem
    textfont=dict(size=12, color='black')  # Konfiguracja czcionki tekstu
))

# Konfiguracja osi i tytułu wykresu
fig.update_layout(
    xaxis_title='Wiek biegaczki',
    yaxis_title='Liczba biegaczek, które ustanowiły rekord osobisty',
    title='Wiek vs Personal Best',
    xaxis=dict(type='linear'),  # Wymuszenie liniowego układu osi X
    yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Wyświetlanie siatki na osi Y
    height= 600,
)

# Zapisywanie wykresu w pliku HTML
fig.write_html('personal_best_graph.html')
fig.show()


In [21]:
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)

# Funkcja do formatowania czasu
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f'{int(hours)}:{int(minutes):02d}:{int(seconds):02d}'

# Wczytanie danych z pliku CSV
data = pd.read_csv('women_marathon.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.split(':').apply(lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))

# Sortowanie wyników względem daty i czasu
data = data.sort_values(by=['Event Date', 'Time'])

# Inicjalizacja pustej listy do przechowywania najlepszych wyników
best_results = []

# Ustalenie daty początkowej
start_date = pd.to_datetime('1968-10-14', format='%Y-%m-%d')

# Przeszukiwanie posortowanych danych w celu znalezienia progresji rekordu świata
current_best_time = float('inf')
for _, row in data.iterrows():
    if row['Event Date'] >= start_date:
        if row['Time'] < current_best_time:
            best_results.append(row)
            current_best_time = row['Time']

# Konwersja listy wyników do DataFrame
best_results = pd.DataFrame(best_results)

# Tworzenie wykresu linii
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=best_results['Event Date'],
    y=best_results['Time'],
    mode='lines',
    name='Progresja rekordu świata - Maraton kobiet'
))

# Ustawienie opisów osi
fig.update_layout(
    xaxis=dict(title='Data wydarzenia'),
    yaxis=dict(title='Czas (hh:mm:ss)')
)

# Ustawienie osi Y
fig.update_yaxes(
    tickvals=[7800, 7680, 7560, 7440, 7320],
    ticktext=['2:10:00', '2:08:00', '2:06:00', '2:04:00', '2:02:00'],
    tickformatstops=[
        dict(dtickrange=[None, 1000], value="%H:%M:%S"),  # Formatowanie na hh:mm:ss dla małych przedziałów
        dict(dtickrange=[1000, None], value="%H:%M")  # Formatowanie na hh:mm dla większych przedziałów
    ]
)

# Dodanie tytułu wykresu
fig.update_layout(
    title="Progresja rekordu świata - Maraton kobiet",
    height= 600,
)

# Zapisywanie wykresu do pliku HTML
fig.write_html("WR_progression.html")

In [22]:
# Read the CSV file
df = pd.read_csv('women_marathon.csv')

# Replace country codes with country names
df['Country'] = df['Country'].replace({
   'GDR': 'Germany',
    'GER': 'Germany',
    'RSA': 'South Africa',
    'NGR': 'Nigeria',
    'POR': 'Portugal',
    'BAR': 'Barbados',
    'ZIM': 'Zimbabwe',
    'BAH': 'Bahamas',
    'ANT': 'Antigua and Barbuda',
    'NED': 'Netherlands',
    'BOT': 'Botswana',
    'CAY': 'Cayman Islands',
    'SKN': 'Saint Kitts and Nevis',
    'AHO': 'Netherlands',
    'SRI': 'Sri Lanka',
    'OMA': 'Oman',
    'SUI': 'Switzerland',
    'MAS': 'Malaysia',
    'KSA': 'Saudi Arabia',
    'IRI': 'Iran',
    'ZAM': 'Zambia',
    'ALG': 'Algeria',
    'ARG': 'Argentina',
    'AUS': 'Australia',
    'AUT': 'Austria',
    'BDI': 'Burundi',
    'BEL': 'Belgium',
    'BEl': 'Belgium',
    'BOL': 'Bolivia',
    'BRA': 'Brazil',
    'BRN': 'Bahrain',
    'CAN': 'Canada',
    'CHI': 'Chile',
    'CHN': 'China',
    'COD': 'Congo',
    'CYP': 'Cyprus',
    'DEN': 'Denmark',
    'DJI': 'Djibouti',
    'ECU': 'Ecuador',
    'ERI': 'Eritrea',
    'ESP': 'Spain',
    'EST': 'Estonia',
    'ETH': 'Ethiopia',
    'FRA': 'France',
    'FRG': 'Germany',
    'GBR': 'United Kingdom',
    'IRL': 'Ireland',
    'ISR': 'Israel',
    'ITA': 'Italy',
    'JPN': 'Japan',
    'KEN': 'Kenya',
    'KOR': 'South Korea',
    'LES': 'Lesotho',
    'MAR': 'Morocco',
    'MDA': 'Moldova',
    'MEX': 'Mexico',
    'MGL': 'Mongolia',
    'NAM': 'Namibia',
    'NOR': 'Norway',
    'NZL': 'New Zealand',
    'PAN': 'Panama',
    'PAR': 'Paraguay',
    'PER': 'Peru',
    'POL': 'Poland',
    'QAT': 'Qatar',
    'RUS': 'Russia',
    'RWA': 'Rwanda',
    'SVK': 'Slovakia',
    'SWE': 'Sweden',
    'TAN': 'Tanzania',
    'TUR': 'Turkey',
    'UGA': 'Uganda',
    'UKR': 'Ukraine',
    'USA': 'United States'
})

# Calculate the number of unique runners per country
country_counts = df.groupby('Country')['Athlete'].nunique().reset_index(name='Unique Runner Count')


# Create the heatmap on the world map
fig = px.choropleth(country_counts, locations='Country', locationmode='country names',
                    color='Unique Runner Count', title='Heatmap of Unique Runners per Country',
                    color_continuous_scale='YlOrRd')

fig.update_layout(
    title_font_size=20,  # Increase the title font size
    font_size=12,  # Set the overall font size
    geo=dict(
        showframe=False,  # Hide the map frame
        showcoastlines=False,  # Hide the coastlines
        projection_type='natural earth'  # Use the natural earth projection
    ),
    title= "Liczba biegaczek z danego kraju",
    height=600,  # Set the height of the graph (in pixels)
    width=1000  # Set the width of the graph (in pixels)
)

fig.write_html("Runners_per_country.html")
fig.show()